<a href="https://colab.research.google.com/github/apurvag4/Apurva_Agarwal/blob/main/CNN_SRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
import tensorflow as tf
import sys
import cv2
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [19]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, ZeroPadding2D, Dropout, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import SGD



In [3]:
Data_dir = "/content/drive/My Drive/Colab Notebooks/FakeNewsDetection/fake-news-detector-master/images_train"
Data_dir_test = "/content/drive/My Drive/Colab Notebooks/FakeNewsDetection/fake-news-detector-master/images_test"

In [4]:
CATEGORIES = ['cover', 'stego']

In [5]:
IMAGE_SIZE = 512
NUM_CHANNELS = 1
PIXEL_DEPTH = 255.
NUM_LABELS = 2
NUM_EPOCHS = 5
batch_size = 16

In [6]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  # cover and stego

        path = os.path.join(Data_dir,category)  # create path to cover and stego
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=cover 1=stego

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMAGE_SIZE, IMAGE_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e: 
                pass
            #except OSError as e:
            #   print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #   print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

100%|██████████| 239/239 [01:57<00:00,  2.03it/s]

409


In [7]:
random.shuffle(training_data)

In [ ]:
validation_data = datagen.flow_from_directory(directory= Data_dir_test,
                                                   target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                   classes = ['cover', 'stego'],
                                                   class_mode = 'binary',
                                                   batch_size = batch_size)

In [8]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)


# print(X[0].reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1))

X = np.array(X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)

In [9]:

##### 0 - Paremeter used in the Batch-Normalization
epsilon = 1e-4


F_0 = tf.cast(tf.constant([[[[-1 / 12.]], [[2 / 12.]], [[-2 / 12.]], [[2 / 12.]], [[-1 / 12.]]],
                           [[[2 / 12.]], [[-6 / 12.]], [[8 / 12.]], [[-6 / 12.]], [[2 / 12.]]],
                           [[[-2 / 12.]], [[8 / 12.]], [[-12 / 12.]], [[8 / 12.]], [[-2 / 12.]]],
                           [[[2 / 12.]], [[-6 / 12.]], [[8 / 12.]], [[-6 / 12.]], [[2 / 12.]]],
                           [[[-1 / 12.]], [[2 / 12.]], [[-2 / 12.]], [[2 / 12.]], [[-1 / 12.]]]]), "float")



In [ ]:
# print(X[0].reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1))

In [ ]:
inputs = X.shape[1:]
model = Sequential()
model.add(Conv2D(F_0, (5, 5), strides= (1,1), padding='SAME', input_shape = inputs))
model.add(tf.math.abs(Conv2D(8, kernel_size= (5,5) ))  # Conv1
model.add(BatchNormalization())
model.add(Activation('Tanh'))
model.add(AveragePooling2D( pool_size=(5,5), strides= (2,2), padding='same'))

model = Sequential()
model.add(Conv2D(16, (5,5)))                              # Conv2
model.add(BatchNormalization())
model.add(Activation('Tanh'))
model.add(AveragePooling2D( pool_size=(5,5), strides= (2,2), padding='same'))

model = Sequential()
model.add(Conv2D(32, (1,1)))                              # Conv3
model.add(BatchNormalization())
model.add(Activation('ReLU'))
model.add(AveragePooling2D( pool_size=(5,5), strides= (2,2), padding='same'))

model = Sequential()
model.add(Conv2D(64, (1,1)))                              # Conv4
model.add(BatchNormalization())
model.add(Activation('ReLU'))
model.add(AveragePooling2D( pool_size=(5,5), strides= (2,2), padding='same'))

model = Sequential()
model.add(Conv2D(128, (1,1)))                              # Conv5
model.add(BatchNormalization())
model.add(Activation('ReLU'))
model.add(AveragePooling2D( pool_size=(5,5), strides= (2,2), padding='same'))

model = Sequential()
model.add(Conv2D(256, (1,1)))                              # Conv6
model.add(BatchNormalization())
model.add(Activation('ReLU'))
model.add(GlobalAveragePooling2D( pool_size=(16,16),strides= (1,1), padding='valid'))

model.add(Dense(1, activation='softmax'))

In [ ]:
opt = SGD(learning_rate=1e-3, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit_generator(generator= (X,y), steps_per_epoch=len( training_data), epochs = 5, validation_data=validation_data, validation_steps=len(validation_data), verbose = 1)

In [ ]:
history.history

In [ ]:
def plot_learningCurve(history, NUM_EPOCHS):
  # Plot training & validation accuracy values
  epoch_range = range(1, NUM_EPOCHS+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningCurve(history, NUM_EPOCHS)